# Подобие Ablation Study

Здесь я сравню полученные в разных моделях результаты

## Грузим результаты

In [ ]:
import pandas as pd

data_folder = '/home/gleb_galagan/tbank_recsys/methods/results_csv/'

greedy_mab_ov = pd.read_csv(data_folder + 'greedy_mab_ov.csv')
greedy_mab_perf = pd.read_csv(data_folder + 'greedy_mab_perf.csv')
greedy_mab_split = pd.read_csv(data_folder + 'greedy_mab_split.csv')


knns_ov = pd.read_csv(data_folder + 'knn_variations_ov.csv')
knns_split = pd.read_csv(data_folder + 'knn_variations_split.csv')
knns_perf = pd.read_csv(data_folder + 'knn_results_perf.csv')


hnsw_ov = pd.read_csv(data_folder + 'hnsw_ov.csv')
hnsw_split = pd.read_csv(data_folder + 'hnsw_split.csv')
hnsw_perf = pd.read_csv(data_folder + 'hnsw_perf.csv')


In [71]:
merged_ov = pd.concat(
    [knns_ov, hnsw_ov, greedy_mab_ov], 
    axis=1
).reset_index()
merged_ov = merged_ov.drop('Unnamed: 0', axis=1).set_index('metric')

In [72]:
# merged_ov.drop('Unnamed: 0', axis=1, inplace=True)
merged_split = pd.concat(
    [knns_split, greedy_mab_split, hnsw_split], 
    axis=1
).reset_index()
merged_split = merged_split.drop('Unnamed: 0', axis=1).set_index('metric')

In [73]:
merged_perf = pd.concat(
    [knns_perf, greedy_mab_perf, hnsw_perf], 
    axis=1
).reset_index()
merged_perf = merged_perf.drop('Unnamed: 0', axis=1).set_index('metric')

Еще я пробовал LightFM, но там прям совсем медленно и проблема с тем чтобы выдать контентные рекомендации такая же как у бустинга. 

- Хочешь использовать только контентные фичи -> получааешь единый список лучших айтемов (дорогой Popular)

- Хочешь чтобы оно сработало -> Мерджишь с интеракциями -> Теряешь холодные айтемы (остаются без рекомендаций)

Но в реализации с LightFM я решил применить гибридный подход. Юзеру отдается 9 горяих айтемов от LightFM, а один айтем отдавался от Greedy MAB, но такой подход работал невероятно долго особенно инференс

In [74]:
# Function to highlight row-wise best/worst
def highlight_row(s):
    is_max = s == s.max()
    is_min = s == s.min()
    return [
        "background-color: green" if v else
        "background-color: salmon" if w else ""
        for v, w in zip(is_max, is_min)
    ]



### Metrics

In [76]:
merged_ov.drop('index', axis=1, inplace=True)
merged_ov.style.apply(highlight_row, axis=1)

,item_knn_title_embeddings,item_knn_image_embedding,item_knn_description_embeddings,hnsw_one,hnsw_multi_recs,hnsw_multi_three,hnsw_multi_four,greedy_mab_ov
metric,,,,,,,,
"('ndcg@10', 'ndcg@10')",0.009000,0.002000,0.010000,0.010000,0.012000,0.012000,0.012000,0.003000
"('recall@10', 'recall@10')",0.009000,0.002000,0.011000,0.010000,0.013000,0.013000,0.012000,0.003000
"('arp@10', 'arp@10')",0.023000,0.014000,0.026000,0.027000,0.037000,0.036000,0.038000,0.035000
"('serendipity@10', 'serendipity@10')",0.006000,0.001000,0.006000,0.006000,0.007000,0.007000,0.007000,0.002000
"('coverage', 'coverage')",0.804000,0.240000,0.687000,0.656000,0.734000,0.729000,0.734000,0.854000


Здесь лучшая модель hnsw_multi_three, по Coverage лидируют Бандиты)

### Perfomance

In [78]:
merged_perf.drop('index', axis=1, inplace=True)

merged_perf.style.apply(highlight_row, axis=1)

,item_knn_cosine_recs,item_knn_title_embeddings,item_knn_image_embedding,item_knn_description_embeddings,greedy_mab_perf,hnsw_one,hnsw_multi_recs,hnsw_multi_three,hnsw_multi_four
metric,,,,,,,,,
"('cold_item_recall', 'cold_item_recall')",0.000000,0.000000,0.000000,0.000000,0.000000,0.003000,0.004000,0.004000,0.004000
"('warm_item_recall', 'warm_item_recall')",0.004000,0.004000,0.001000,0.004000,0.002000,0.004000,0.005000,0.005000,0.005000
"('total_cold_items_in_gt', 'total_cold_items_in_gt')",0.000000,0.000000,0.000000,0.000000,187372.000000,187372.000000,187372.000000,187372.000000,187372.000000
"('total_warm_items_in_gt', 'total_warm_items_in_gt')",3134593.000000,3134593.000000,3134593.000000,3134593.000000,2947221.000000,2947221.000000,2947221.000000,2947221.000000,2947221.000000
"('cold_items_recommended', 'cold_items_recommended')",0.000000,0.000000,0.000000,0.000000,29.000000,631.000000,789.000000,800.000000,762.000000
"('warm_items_recommended', 'warm_items_recommended')",13152.000000,11641.000000,3017.000000,13158.000000,5358.000000,12178.000000,15452.000000,14844.000000,13883.000000


Лучшие метрики на холодных и горячих у hnsw_multi_three, здесь больше всего холодных рекомендаций

### User by split

In [77]:
merged_split.drop('index', axis=1, inplace=True)

merged_split.style.apply(highlight_row, axis=1)

,item_knn_title_embeddings,item_knn_image_embedding,item_knn_description_embeddings,greedy_mab_split,hnsw_one,hnsw_multi_recs,hnsw_multi_three,hnsw_multi_four
metric,,,,,,,,
"('ndcg@10_pure_cold', 'ndcg@10_pure_cold')",0.006000,0.000000,0.010000,0.000000,0.013000,0.014000,0.016000,0.015000
"('recall@10_pure_cold', 'recall@10_pure_cold')",0.013000,0.001000,0.025000,0.000000,0.024000,0.024000,0.028000,0.025000
"('arp@10_pure_cold', 'arp@10_pure_cold')",0.018000,0.011000,0.018000,0.034000,0.020000,0.027000,0.026000,0.028000
"('serendipity@10_pure_cold', 'serendipity@10_pure_cold')",0.001000,0.000000,0.003000,0.000000,0.002000,0.003000,0.003000,0.003000
"('ndcg@10_mixed', 'ndcg@10_mixed')",0.012000,0.004000,0.013000,0.006000,0.013000,0.016000,0.016000,0.015000
"('recall@10_mixed', 'recall@10_mixed')",0.004000,0.001000,0.005000,0.002000,0.005000,0.007000,0.007000,0.006000
"('arp@10_mixed', 'arp@10_mixed')",0.024000,0.013000,0.025000,0.035000,0.026000,0.036000,0.035000,0.038000
"('serendipity@10_mixed', 'serendipity@10_mixed')",0.010000,0.003000,0.011000,0.005000,0.010000,0.013000,0.013000,0.011000
"('ndcg@10_pure_warm', 'ndcg@10_pure_warm')",0.008000,0.002000,0.009000,0.003000,0.009000,0.011000,0.011000,0.011000


Также hnsw_multi_three лучшие метрики на холодных и горячих

## Future work

- Из простого для каждого юзера конкатенировать описания прочитанных им книг (или просто тайтлов), далее получить эмбеддинг этого текста, это будет вектор предпочтений юзера. Этот эмбеддинг уже можно по метрике сравнивать с эмбеддингами описаний или названий книг

- При проблеме холодного старта у айтемов сразу подумал про Semantic IDs, времени их реализовать к сожалению не было, но если бы время было начал бы строить их с помощью RQ-VAE аналагично тому как представлено в статье [Better Generalization with Semantic IDs: A Case Study in Ranking for Recommendations](https://arxiv.org/abs/2306.08121)
Но еще бы я попробовал сделать SIDs на кластеризации как тут [«Semantic IDs: архитектура и наш опыт внедрения» — Александр Тришин WB](https://www.youtube.com/watch?v=kfiG2ChpM34&t=1473s&ab_channel=WBTech)

- Еще во время отбора я вдохновлялся статьей [Сама статья](https://arxiv.org/abs/2502.18965), [Тех репорт 1](https://arxiv.org/abs/2506.13695), [Тех репорт 2](https://arxiv.org/abs/2508.20900). Крутой момент то что мы избавляемся от этапа ранжирования где тот же самый бустинг может попасть в feedback loop и убрать наши новые непопулярные айтемы. Но трансформер у меня точно не заведется, Даже бустинг с эмбеддингами не очень быстро считался

**Спасибо за внимание, надеюсь встретимся на собеседовании а затем в Сириусе**
